In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
FILE_PATH = r"D:\aistudy\metacode_study\06-DocumentLoader\data\KDS140000Structure.pdf"
print(FILE_PATH)

D:\aistudy\metacode_study\06-DocumentLoader\data\KDS140000Structure.pdf


In [4]:
"""
PDF → LaTeX 변환 파이프라인 (메모리 효율적)
수식이 포함된 PDF를 LaTeX로 변환하여 RAG에 활용

특징:
1. 이미지를 디스크에 저장하지 않고 메모리에서 처리
2. 각 컴포넌트를 독립적으로 사용 가능
3. 스트리밍 방식으로 페이지별 처리
"""

import torch
from PIL import Image
from io import BytesIO
import fitz  # PyMuPDF
from typing import Generator, List, Dict, Optional
from dataclasses import dataclass


# ============================================================
# 1. 이미지 추출 컴포넌트
# ============================================================

@dataclass
class PageImage:
    """페이지 이미지 데이터"""
    page_num: int
    image: Image.Image
    width: int
    height: int
    
    def __post_init__(self):
        self.width, self.height = self.image.size


class PDFImageExtractor:
    """PDF에서 이미지를 추출하는 컴포넌트"""
    
    def __init__(self, dpi: int = 300):
        """
        Args:
            dpi: 이미지 해상도 (144=zoom 2.0, 216=zoom 3.0, 300=zoom 4.17)
        """
        self.dpi = dpi
        self.zoom = dpi / 72.0  # 72 DPI가 기본
    
    def extract_page(self, pdf_path: str, page_num: int) -> PageImage:
        """
        단일 페이지를 이미지로 변환 (메모리에만 보관)
        
        Args:
            pdf_path: PDF 파일 경로
            page_num: 페이지 번호 (1부터 시작)
        
        Returns:
            PageImage 객체
        """
        doc = fitz.open(pdf_path)
        
        try:
            # 페이지 추출 (0-based index)
            page = doc[page_num - 1]
            
            # 이미지로 렌더링
            mat = fitz.Matrix(self.zoom, self.zoom)
            pix = page.get_pixmap(matrix=mat, alpha=False)
            
            # PIL Image로 변환 (디스크 저장 없음)
            img_bytes = pix.tobytes("png")
            image = Image.open(BytesIO(img_bytes))
            
            return PageImage(
                page_num=page_num,
                image=image,
                width=image.size[0],
                height=image.size[1]
            )
        
        finally:
            doc.close()
    
    def extract_pages(
        self, 
        pdf_path: str, 
        page_range: Optional[tuple] = None
    ) -> Generator[PageImage, None, None]:
        """
        여러 페이지를 스트리밍 방식으로 추출
        
        Args:
            pdf_path: PDF 파일 경로
            page_range: (시작, 끝) 페이지 번호. None이면 전체
        
        Yields:
            PageImage 객체
        """
        doc = fitz.open(pdf_path)
        
        try:
            total_pages = len(doc)
            
            if page_range is None:
                start, end = 1, total_pages
            else:
                start, end = page_range
                end = min(end, total_pages)
            
            print(f"총 {total_pages}페이지 중 {start}~{end} 페이지 처리")
            
            for page_num in range(start, end + 1):
                page = doc[page_num - 1]
                
                mat = fitz.Matrix(self.zoom, self.zoom)
                pix = page.get_pixmap(matrix=mat, alpha=False)
                
                img_bytes = pix.tobytes("png")
                image = Image.open(BytesIO(img_bytes))
                
                yield PageImage(
                    page_num=page_num,
                    image=image,
                    width=image.size[0],
                    height=image.size[1]
                )
        
        finally:
            doc.close()


# ============================================================
# 2. OCR 컴포넌트
# ============================================================

@dataclass
class OCRResult:
    """OCR 결과 데이터"""
    page_num: int
    text: str
    confidence: Optional[float] = None
    processing_time: Optional[float] = None


class NougatOCR:
    """Nougat을 사용한 OCR 컴포넌트"""
    
    def __init__(
        self, 
        model_name: str = "facebook/nougat-base",
        use_gpu: bool = True
    ):
        """
        Args:
            model_name: Nougat 모델 이름
            use_gpu: GPU 사용 여부
        """
        self.model_name = model_name
        self.use_gpu = use_gpu and torch.cuda.is_available()
        self.model = None
        self.device = "cuda" if self.use_gpu else "cpu"
    
    def load_model(self):
        """모델을 메모리에 로드"""
        if self.model is not None:
            return
        
        print(f"Nougat 모델 로딩 중... (device: {self.device})")
        
        try:
            from nougat import NougatModel
            self.model = NougatModel.from_pretrained(self.model_name)
            self.model = self.model.to(self.device)
            self.model.eval()
            print("✓ 모델 로딩 완료")
        
        except ImportError:
            raise ImportError(
                "Nougat이 설치되지 않았습니다.\n"
                "설치: pip install nougat-ocr"
            )
    
    def process_image(self, image: Image.Image) -> str:
        """
        이미지에서 LaTeX 텍스트 추출
        
        Args:
            image: PIL Image 객체
        
        Returns:
            LaTeX/Markdown 형식의 텍스트
        """
        import time
        from nougat.postprocessing import markdown_compatible
        
        if self.model is None:
            self.load_model()
        
        start_time = time.time()
        
        try:
            with torch.no_grad():
                # inference 호출 (메모리에서 직접 처리)
                output = self.model.inference(image=image)
                output = markdown_compatible(output)
            
            processing_time = time.time() - start_time
            print(f"  OCR 완료 ({processing_time:.2f}초)")
            
            return output
        
        except Exception as e:
            print(f"  OCR 오류: {e}")
            # CLI fallback이 필요한 경우
            raise RuntimeError(
                f"OCR 처리 실패: {e}\n"
                "CLI 사용을 고려하세요: nougat pdf_file.pdf -o output"
            )
    
    def process_page_image(self, page_image: PageImage) -> OCRResult:
        """
        PageImage 객체를 처리
        
        Args:
            page_image: PageImage 객체
        
        Returns:
            OCRResult 객체
        """
        import time
        
        print(f"페이지 {page_image.page_num} OCR 처리 중...")
        start_time = time.time()
        
        text = self.process_image(page_image.image)
        processing_time = time.time() - start_time
        
        return OCRResult(
            page_num=page_image.page_num,
            text=text,
            processing_time=processing_time
        )
    
    def unload_model(self):
        """모델을 메모리에서 제거"""
        if self.model is not None:
            del self.model
            self.model = None
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            print("✓ 모델 언로드 완료")


# ============================================================
# 3. 문서 로더 컴포넌트
# ============================================================

@dataclass
class Document:
    """RAG에서 사용할 문서 객체"""
    page_content: str
    metadata: Dict
    
    def __repr__(self):
        preview = self.page_content[:100].replace('\n', ' ')
        return f"Document(page={self.metadata.get('page')}, content='{preview}...')"


class PDFLaTeXLoader:
    """PDF를 LaTeX로 변환하여 로드하는 컴포넌트"""
    
    def __init__(
        self,
        extractor: PDFImageExtractor,
        ocr: NougatOCR
    ):
        """
        Args:
            extractor: PDFImageExtractor 인스턴스
            ocr: NougatOCR 인스턴스
        """
        self.extractor = extractor
        self.ocr = ocr
    
    def load_page(self, pdf_path: str, page_num: int) -> Document:
        """
        단일 페이지를 로드
        
        Args:
            pdf_path: PDF 파일 경로
            page_num: 페이지 번호 (1부터 시작)
        
        Returns:
            Document 객체
        """
        # 1. 이미지 추출 (메모리)
        page_image = self.extractor.extract_page(pdf_path, page_num)
        
        # 2. OCR 처리 (메모리)
        ocr_result = self.ocr.process_page_image(page_image)
        
        # 3. Document 생성
        document = Document(
            page_content=ocr_result.text,
            metadata={
                'source': pdf_path,
                'page': page_num,
                'width': page_image.width,
                'height': page_image.height,
                'processing_time': ocr_result.processing_time
            }
        )
        
        return document
    
    def load_pages(
        self, 
        pdf_path: str, 
        page_range: Optional[tuple] = None
    ) -> Generator[Document, None, None]:
        """
        여러 페이지를 스트리밍 방식으로 로드
        
        Args:
            pdf_path: PDF 파일 경로
            page_range: (시작, 끝) 페이지 번호
        
        Yields:
            Document 객체
        """
        # OCR 모델 로드 (한 번만)
        self.ocr.load_model()
        
        try:
            # 페이지별로 스트리밍 처리
            for page_image in self.extractor.extract_pages(pdf_path, page_range):
                # OCR 처리
                ocr_result = self.ocr.process_page_image(page_image)
                
                # Document 생성
                document = Document(
                    page_content=ocr_result.text,
                    metadata={
                        'source': pdf_path,
                        'page': page_image.page_num,
                        'width': page_image.width,
                        'height': page_image.height,
                        'processing_time': ocr_result.processing_time
                    }
                )
                
                yield document
        
        finally:
            # 메모리 정리
            self.ocr.unload_model()
    
    def load_all(
        self, 
        pdf_path: str, 
        page_range: Optional[tuple] = None
    ) -> List[Document]:
        """
        모든 페이지를 한 번에 로드 (메모리 사용량 주의)
        
        Args:
            pdf_path: PDF 파일 경로
            page_range: (시작, 끝) 페이지 번호
        
        Returns:
            Document 객체 리스트
        """
        return list(self.load_pages(pdf_path, page_range))


# ============================================================
# 4. 사용 예제
# ============================================================

def example_single_page(pdf_path: str, page_num: int = 1, dpi: int = 300, use_gpu: bool = True):
    """
    단일 페이지 처리 예제
    
    Args:
        pdf_path: PDF 파일 경로
        page_num: 페이지 번호 (기본값: 1)
        dpi: 이미지 해상도 (기본값: 300)
        use_gpu: GPU 사용 여부 (기본값: True)
    
    Returns:
        Document 객체
    """
    print("\n" + "="*60)
    print("예제 1: 단일 페이지 처리")
    print("="*60 + "\n")
    
    # 컴포넌트 초기화
    extractor = PDFImageExtractor(dpi=dpi)
    extractor.extract_page(pdf_path=pdf_path, page_num=page_num)
    ocr = NougatOCR(use_gpu=use_gpu)
    loader = PDFLaTeXLoader(extractor, ocr)
    
    # 페이지 로드
    document = loader.load_page(pdf_path, page_num)
    
    print(f"\n문서 정보:")
    print(f"  페이지: {document.metadata['page']}")
    print(f"  크기: {document.metadata['width']}x{document.metadata['height']}")
    print(f"  처리시간: {document.metadata['processing_time']:.2f}초")
    print(f"\n내용 미리보기:")
    print(document.page_content[:500])
    
    return document


def example_streaming():
    """스트리밍 방식 처리 예제 (메모리 효율적)"""
    print("\n" + "="*60)
    print("예제 2: 스트리밍 방식 처리 (메모리 효율적)")
    print("="*60 + "\n")
    
    # 컴포넌트 초기화
    extractor = PDFImageExtractor(dpi=300)
    ocr = NougatOCR(use_gpu=True)
    loader = PDFLaTeXLoader(extractor, ocr)
    
    # 페이지별로 처리 (메모리에 한 번에 하나만 보관)
    pdf_path = "your_file.pdf"
    page_range = (1, 5)  # 1-5페이지
    
    for document in loader.load_pages(pdf_path, page_range):
        print(f"\n페이지 {document.metadata['page']} 로드 완료")
        print(f"  내용 길이: {len(document.page_content)} 문자")
        
        # 여기서 바로 벡터DB에 저장 가능
        # vector_db.add_document(document)
        
        # 또는 파일로 저장
        with open(f"page_{document.metadata['page']}.md", 'w', encoding='utf-8') as f:
            f.write(document.page_content)


def example_batch():
    """일괄 처리 예제"""
    print("\n" + "="*60)
    print("예제 3: 일괄 처리")
    print("="*60 + "\n")
    
    # 컴포넌트 초기화
    extractor = PDFImageExtractor(dpi=300)
    ocr = NougatOCR(use_gpu=True)
    loader = PDFLaTeXLoader(extractor, ocr)
    
    # 모든 페이지 로드
    pdf_path = "your_file.pdf"
    documents = loader.load_all(pdf_path, page_range=(1, 3))
    
    print(f"\n총 {len(documents)}개 문서 로드 완료")
    
    for doc in documents:
        print(f"  - {doc}")
    
    return documents


def example_with_langchain():
    """LangChain과 통합 예제"""
    print("\n" + "="*60)
    print("예제 4: LangChain RAG 파이프라인")
    print("="*60 + "\n")
    
    # 컴포넌트 초기화
    extractor = PDFImageExtractor(dpi=300)
    ocr = NougatOCR(use_gpu=True)
    loader = PDFLaTeXLoader(extractor, ocr)
    
    pdf_path = "your_file.pdf"
    
    # 스트리밍으로 처리하면서 LangChain에 통합
    documents_for_langchain = []
    
    for document in loader.load_pages(pdf_path, page_range=(1, 10)):
        # LangChain Document 형식으로 변환 가능
        documents_for_langchain.append(document)
    
    # 텍스트 스플리터 적용 예제
    print("\n텍스트 스플릿 준비 완료")
    print(f"총 {len(documents_for_langchain)}개 문서")
    
    # 다음 단계:
    # from langchain.text_splitter import RecursiveCharacterTextSplitter
    # splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
    # splits = splitter.split_documents(documents_for_langchain)
    
    return documents_for_langchain


if __name__ == "__main__":
    # 사용하고 싶은 예제 선택
    
    # 1. 단일 페이지
    # example_single_page()
    
    # 2. 스트리밍 (권장 - 메모리 효율적)
    # example_streaming()
    
    # 3. 일괄 처리
    # example_batch()
    
    # 4. LangChain 통합
    # example_with_langchain()
    
    pass

In [6]:
doc = example_single_page(FILE_PATH, page_num=34, use_gpu=True)


예제 1: 단일 페이지 처리

페이지 34 OCR 처리 중...


d:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:108: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  super().__init__(always_apply=always_apply, p=p)
d:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:42: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  super().__init__(always_apply=always_apply, p=p)
d:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:76: UserWarning: Argument(s) 'always_apply' are not valid for transform BasicTransform
  super().__init__(always_apply=always_apply, p=p)
d:\aistudy\metacode_study\tenv\Lib\site-packages\nougat\transforms.py:123: UserWarning: Argument(s) 'cval' are not valid for transform Affine
  alb.Affine(shear={"x": (0, 3), "y": (-3, 0)}, cval=(255, 255, 255), p=0.03),
d:\aistudy\metacode_study\tenv\Lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transfor

ValueError: 1 validation error for InitSchema
compression_type
  Input should be 'jpeg' or 'webp' [type=literal_error, input_value=95, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error